In [1]:
import keras
import tensorflow as tf

from keras.applications import ResNet50, VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Flatten, Dense, Input, Dropout
from keras.layers import Conv2D, ZeroPadding2D
from keras.layers import MaxPooling2D
from keras.metrics import top_k_categorical_accuracy
from keras.models import Model
from keras.optimizers import SGD
from keras import backend as K

Using TensorFlow backend.


In [2]:
train_datagen = ImageDataGenerator(
        samplewise_center=True,
        samplewise_std_normalization=True,
        horizontal_flip=True,
        vertical_flip=True)

test_datagen = ImageDataGenerator()

In [3]:
train_generator = train_datagen.flow_from_directory(
    '../data/data_small_train/',     
    class_mode='categorical',           
    target_size=(224, 224),
    batch_size=50)

validation_generator = test_datagen.flow_from_directory(
    '../data/data_small_test/',  
    class_mode='categorical',
    target_size=(224, 224),
    batch_size=50)

Found 2509 images belonging to 25 classes.
Found 614 images belonging to 25 classes.


In [4]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=True)

In [13]:
#Get back the convolutional part of a VGG network trained on ImageNet
model_resnet50 = ResNet50(weights='imagenet', include_top=False)
model_resnet50.get_layer('conv1').trainable = False
#model_vgg16_conv.layers[2].trainable = False
#model_vgg16_conv.layers[4].trainable = False
#model_vgg16_conv.layers[5].trainable = False
#model_vgg16_conv.layers[7].trainable = False
#model_vgg16_conv.layers[8].trainable = False
#model_vgg16_conv.layers[9].trainable = False
#model_vgg16_conv.layers[11].trainable = False
#model_vgg16_conv.layers[12].trainable = False
#model_vgg16_conv.layers[13].trainable = False
#model_vgg16_conv.layers[15].trainable = False
#model_vgg16_conv.layers[16].trainable = False
#model_vgg16_conv.layers[18].trainable = False
#
#input = Input(shape=(224, 224, 3), name='image_input')
##
#output_restnet50 = model_resnet50(input)
##
#x = Flatten(name='flatten')(output_restnet50)
#x = Dense(4096, activation='relu', name='fc1')(x)
#x = Dense(4096, activation='relu', name='fc2')(x)
#x = Dense(25, activation='softmax', name='predictions')(x)
#
#model = Model(inputs = input, outputs=x)
#
#model.summary()
model_resnet50.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_9 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
zero_padding2d_9 (ZeroPadding2D) (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, None, None, 64 9472                                         
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, None, None, 64 256                                          
___________________________________________________________________________________________

In [8]:
def top_5_categorical_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=5)

In [9]:
sgd = SGD(lr=0.01, decay=0.0005, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy', top_5_categorical_accuracy])

In [10]:
model.fit_generator(
    generator=train_generator,
    steps_per_epoch=50,
    validation_data=validation_generator,
    validation_steps=12,
    epochs=100,
    callbacks=[tbCallBack],
    verbose=2)

Epoch 1/100
101s - loss: 2.3235 - acc: 0.3340 - top_5_categorical_accuracy: 0.6568 - val_loss: 3.3836 - val_acc: 0.2850 - val_top_5_categorical_accuracy: 0.5683
Epoch 2/100
94s - loss: 1.4629 - acc: 0.5755 - top_5_categorical_accuracy: 0.8594 - val_loss: 3.9388 - val_acc: 0.2039 - val_top_5_categorical_accuracy: 0.4521
Epoch 3/100
93s - loss: 1.0456 - acc: 0.6818 - top_5_categorical_accuracy: 0.9242 - val_loss: 6.9457 - val_acc: 0.0798 - val_top_5_categorical_accuracy: 0.2766
Epoch 4/100
93s - loss: 0.7386 - acc: 0.7818 - top_5_categorical_accuracy: 0.9596 - val_loss: 10.7839 - val_acc: 0.0426 - val_top_5_categorical_accuracy: 0.1915
Epoch 5/100
93s - loss: 0.5922 - acc: 0.8174 - top_5_categorical_accuracy: 0.9736 - val_loss: 13.3526 - val_acc: 0.0496 - val_top_5_categorical_accuracy: 0.1738
Epoch 6/100


KeyboardInterrupt: 